## In this notebook, we will implement IOU that are used in many computer vision applications

1. Batch/Vectorize implementation for efficiency

### References
https://pytorch.org/vision/stable/_modules/torchvision/ops/boxes.html#box_iou
https://github.com/kuangliu/torchcv/blob/master/torchcv/utils/box.py \
https://github.com/kuangliu/torchcv/blob/master/torchcv/utils/box.py

In [1]:
import torch
from torch import Tensor

In [6]:
def box_area(boxes:Tensor)->Tensor:
    """
    Calculates Area of the given boxes
    boxes:Tensor[minibatch, 4] where coordinates are in xyxy format
    return:Tensor[minibatch]
    """
    return (boxes[:,2] - boxes[:,0]) * (boxes[:,3]-boxes[:,1]) 



def box_iou(boxes1:Tensor, boxes2:Tensor)->Tensor:
    """
    inputs: Tensors [minibatch ,4] in xyxy format
    calculates the IOU of all boxes against all boxes
    if boxes1 contains N boxes and boxes2 contains M boxes
    return Tensor [N, M, IOU]
    """

    # calculate the intersection top left
    # create a extra axis so boxes2 will be broadcast N times
    # torch maxium will be the elementwise comparison

    area_1 = box_area(boxes1)
    area_2 = box_area(boxes2)

    top_left_coords = torch.maximum(boxes1[:,None,:2], boxes2[:,:2])
    btm_right_coords = torch.minimum(boxes1[:, None,2:4], boxes2[:,2:4])

    height_width = torch.clamp(btm_right_coords-top_left_coords, min=0)     #[N,M,2] where 2 is width, height eg (2,4) or (-3,0), any negative value indicates no intersection
    intersection = height_width[:,:,0] * height_width[:,:,1]  #[N,M]

    area = area_1[:,None] + area_2 - intersection # [N,M] -[N,M]

    return area/intersection
    
    
    